# DWI preprocessing
## using MRTRIX, FSL and ANTS
### by Michael Paquette
#### Notes from Dec 9th

In this serie of bash notebook, I will be describing a basic DWI data preprocessing pipeline.

I will try to stick to this (loose) general format to describe each step:  

- What is the artefact we are trying to correct or the transformation we are trying to achieve  
- Why does this happen  
- What would happen if we didnt correct/modify 
- Why is this step here (somewhat arbitrary or specific ordering)  
- Important parameters of the tool (and which one are likely to need finetuning)  
- What to look for when doing QC (quality control)

## Bash refresher

Here are a few bash commands that are enough to navigate through a typical data folder structure.

pwd  
stands for "print working directory", it prints out the location (path) where you currently are.

In [86]:
# We move to a specific folder and print the current location
cd /data/pt_02586/
pwd

# We move to a specific folder and print the current location
cd /data/pt_02586/
pwd
/data/pt_02586


: 1

ls  
stands for "list", it prints out the content (files and directories) of a path. You can specify a path "ls /path/to/somewhere" or leave in blank to show current directory. 

In [87]:
# list stuff in current directory
ls

# list stuff in current directory
ls
"DTI_analysis.rar"*	"sub_03"/  "sub_08"/  "sub_13"/  "sub_18"/
"processing_tutorial"/	"sub_04"/  "sub_09"/  "sub_14"/  "sub_19"/
"slicesdir"/		"sub_05"/  "sub_10"/  "sub_15"/  "sub_20"/
"sub_01"/		"sub_06"/  "sub_11"/  "sub_16"/  "sub_21"/
"sub_02"/		"sub_07"/  "sub_12"/  "sub_17"/  "tmp"/


: 1

In [88]:
# list stuff at specific path
ls /data/pt_02586/

# list stuff at specific path
ls /data/pt_02586/
"DTI_analysis.rar"*	"sub_03"/  "sub_08"/  "sub_13"/  "sub_18"/
"processing_tutorial"/	"sub_04"/  "sub_09"/  "sub_14"/  "sub_19"/
"slicesdir"/		"sub_05"/  "sub_10"/  "sub_15"/  "sub_20"/
"sub_01"/		"sub_06"/  "sub_11"/  "sub_16"/  "sub_21"/
"sub_02"/		"sub_07"/  "sub_12"/  "sub_17"/  "tmp"/


: 1

Most bash commands have other functionality that can be accessed by giving the command some extra arguments called "flags". For instance, the "ls" command has a very usefull flag "-l" which stands for the "long" version of "ls" with more info 

In [89]:
ls -l /data/pt_02586/

ls -l /data/pt_02586/
total 2093122
-rwxrwx--- 1 anwander users 2253253007 Nov 26 10:32 "DTI_analysis.rar"*
drwxrwx--- 3 paquette users          4 Dec 13 14:19 "processing_tutorial"/
drwxrwx--- 2 anwander users         54 Nov 26 11:37 "slicesdir"/
drwxrwx--- 4 anwander users         16 Dec  9 16:48 "sub_01"/
drwxrwx--- 3 anwander users          9 Nov 26 11:26 "sub_02"/
drwxrwx--- 3 anwander users          9 Nov 26 11:27 "sub_03"/
drwxrwx--- 3 anwander users          9 Nov 26 11:27 "sub_04"/
drwxrwx--- 3 anwander users          9 Nov 26 11:27 "sub_05"/
drwxrwx--- 3 anwander users          9 Nov 26 11:28 "sub_06"/
drwxrwx--- 3 anwander users          9 Nov 26 11:28 "sub_07"/
drwxrwx--- 3 anwander users          9 Nov 26 11:28 "sub_08"/
drwxrwx--- 3 anwander users          9 Nov 26 11:29 "sub_09"/
drwxrwx--- 3 anwander users          9 Nov 26 11:29 "sub_10"/
drwxrwx--- 3 anwander users          9 Nov 26 11:29 "sub_11"/
drwxrwx--- 3 anwander users          9 Nov 26 11:30 "sub_12"/
drwxrwx-

: 1

Personally, I almost exclusively use "ls -lhtr"  
the "-l" flag is the long output  
the  "-h" flag is the "human readable file size" flag, it mean that instead of printing something like "19455325" bytes as size, it would convert it to "18.554MB"  
the "-t" flag sorts files by "last modified date" instead of the default alphabetical, newer files are at the top of the ouput.  
the "-r" flag reverses the order the files are printed, making the newer one at the bottom in this case.  
"ls -lhtr" could be written with a different flag ordering or by separating the flags "ls -l -t -r -h" but conviniently 1 letter long flags can be stacked

In [90]:
ls -lhtr

ls -lhtr
total 2.0G
-rwxrwx--- 1 anwander users 2.1G Nov 26 10:32 "DTI_analysis.rar"*
drwxrwx--- 3 anwander users    9 Nov 26 11:26 "sub_02"/
drwxrwx--- 3 anwander users    9 Nov 26 11:27 "sub_03"/
drwxrwx--- 3 anwander users    9 Nov 26 11:27 "sub_04"/
drwxrwx--- 3 anwander users    9 Nov 26 11:27 "sub_05"/
drwxrwx--- 3 anwander users    9 Nov 26 11:28 "sub_06"/
drwxrwx--- 3 anwander users    9 Nov 26 11:28 "sub_07"/
drwxrwx--- 3 anwander users    9 Nov 26 11:28 "sub_08"/
drwxrwx--- 3 anwander users    9 Nov 26 11:29 "sub_09"/
drwxrwx--- 3 anwander users    9 Nov 26 11:29 "sub_10"/
drwxrwx--- 3 anwander users    9 Nov 26 11:29 "sub_11"/
drwxrwx--- 3 anwander users    9 Nov 26 11:30 "sub_12"/
drwxrwx--- 3 anwander users    9 Nov 26 11:30 "sub_13"/
drwxrwx--- 3 anwander users    9 Nov 26 11:30 "sub_14"/
drwxrwx--- 3 anwander users    9 Nov 26 11:31 "sub_15"/
drwxrwx--- 3 anwander users    9 Nov 26 11:31 "sub_16"/
drwxrwx--- 3 anwander users    9 Nov 26 11:32 "sub_17"/
drwxrwx--- 3 anwan

: 1

In [91]:
# it the same!
ls -l -t -r -h

# it the same!
ls -l -t -r -h
total 2.0G
-rwxrwx--- 1 anwander users 2.1G Nov 26 10:32 "DTI_analysis.rar"*
drwxrwx--- 3 anwander users    9 Nov 26 11:26 "sub_02"/
drwxrwx--- 3 anwander users    9 Nov 26 11:27 "sub_03"/
drwxrwx--- 3 anwander users    9 Nov 26 11:27 "sub_04"/
drwxrwx--- 3 anwander users    9 Nov 26 11:27 "sub_05"/
drwxrwx--- 3 anwander users    9 Nov 26 11:28 "sub_06"/
drwxrwx--- 3 anwander users    9 Nov 26 11:28 "sub_07"/
drwxrwx--- 3 anwander users    9 Nov 26 11:28 "sub_08"/
drwxrwx--- 3 anwander users    9 Nov 26 11:29 "sub_09"/
drwxrwx--- 3 anwander users    9 Nov 26 11:29 "sub_10"/
drwxrwx--- 3 anwander users    9 Nov 26 11:29 "sub_11"/
drwxrwx--- 3 anwander users    9 Nov 26 11:30 "sub_12"/
drwxrwx--- 3 anwander users    9 Nov 26 11:30 "sub_13"/
drwxrwx--- 3 anwander users    9 Nov 26 11:30 "sub_14"/
drwxrwx--- 3 anwander users    9 Nov 26 11:31 "sub_15"/
drwxrwx--- 3 anwander users    9 Nov 26 11:31 "sub_16"/
drwxrwx--- 3 anwander users    9 Nov 26 11:32 "sub_17

: 1

cd  
stands for "change directory", its allows you to move you location. You can specify a full path or you can specify a path relative to you current location.

In [92]:
# cd with specific path
cd /data/hu_paquette/Documents
# we are now there
pwd

# cd with specific path
cd /data/hu_paquette/Documents
# we are now there
pwd
/data/hu_paquette/Documents


: 1

In [93]:
# inspect what is there
ls

# inspect what is there
ls
"battocchio_miccai_graph.pdf"*		     "MATLAB"/
"courses"/				     "receipt_don_scc.pdf"*
"data_timestamps.txt"*			     "test_rotated_bvec.txt"*
"diffusion_parameter_exvivo_citations.txt"*  "waveform"/
"hippocampus_exvivo_temperatue_paper.png"*


: 1

In [94]:
# move with cd using relative path
cd courses
pwd

# move with cd using relative path
cd courses
pwd
/data/hu_paquette/Documents/courses


: 1

mv  
Stands for "move", we use it to move files "mv /old/path /new/path". Like always the paths can be full paths or paths relative to the current location.  
This also how you rename files in bash, you "move" them to the same location but wth a different name.  
If you leave the path black after a "/", the file will keep its original name when it move to that folder.  
You can also write "./" to refer to the current folder for example "mv /path/to/file.txt ./" will move that file to the current folder with the name file.txt

In [95]:
ls
#
# we'll move back forth that pdf file 
mv Short-refcard.pdf /data/hu_paquette/Documents/
#
# we see that the file is gone
ls
#
# because it is here now
ls /data/hu_paquette/Documents/
# 
# and we move it back
mv /data/hu_paquette/Documents/Short-refcard.pdf ./

ls
"audio_python_illinois"/	       "edx_ph525.1x"/
"dataanalysisforthelifesciences.pdf"*  "Short-refcard.pdf"*
#
# we'll move back forth that pdf file 
mv Short-refcard.pdf /data/hu_paquette/Documents/
#
# we see that the file is gone
ls
"audio_python_illinois"/	       "edx_ph525.1x"/
"dataanalysisforthelifesciences.pdf"*
#
# because it is here now
ls /data/hu_paquette/Documents/
"battocchio_miccai_graph.pdf"*		     "MATLAB"/
"courses"/				     "receipt_don_scc.pdf"*
"data_timestamps.txt"*			     "Short-refcard.pdf"*
"diffusion_parameter_exvivo_citations.txt"*  "test_rotated_bvec.txt"*
"hippocampus_exvivo_temperatue_paper.png"*   "waveform"/
# 
# and we move it back
mv /data/hu_paquette/Documents/Short-refcard.pdf ./


: 1

cp  
stands for "copy", works the same has mv but makes a copy at the new location without removing it from the original location.

cat  
Stands for "concatenate", its a usefull simple command for print textfiles. If you list multiple textfiles after the "cat" they all get printed one after the other (therefore the name concatenate).

In [96]:
# first we move back here
cd /data/pt_02586/sub_01/
ls

# first we move back here
cd /data/pt_02586/sub_01/
ls
"bvals.txt"*
"bvecs.txt"*
"DICOM"/
"DTI_MP_64_2x2x2_s2_p2_6-8_20210910130531_9.bval"*
"DTI_MP_64_2x2x2_s2_p2_6-8_20210910130531_9.bvec"*
"DTI_MP_64_2x2x2_s2_p2_6-8_20210910130531_9.json"*
"DTI_MP_64_2x2x2_s2_p2_6-8_20210910130531_9.nii.gz"*
"eddy_config.txt"*
"eddy_ind.txt"*
"pe_table.txt"*
"preprocessing"/
"t1_mprage_sag_1x1x1_p2_64ch_20210910130531_3.json"*
"t1_mprage_sag_1x1x1_p2_64ch_20210910130531_3.nii.gz"*
"tmp.nii"*


: 1

In [97]:
cat bvals.txt

cat bvals.txt
0 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 0 0 0 0 0 0


: 1

mkdir  
stands for "make directory", creates a new directory at the path specified, can be a full path or a relative one to you location.

rm  
stands for "remove", it deletes files.  
#### rm is permanent and irreversible, there is no way to recover the files, so if deleting multiple files at once, its good pratice to "ls" the files you want to delete before to double check that they are the right files

Variables in bash  
variable in bash are very usefull for data preprocessing.  
We can store important paths like the location of the data or the folder we are doing the processing for a specific subject.  

In [98]:
# save the root data directory in a variable
ROOTDIR='/data/pt_02586/'

# save the root data directory in a variable
ROOTDIR='/data/pt_02586/'


: 1

In [99]:
# The variable can be used by using a $ sign 
# in front of the name
# echo is a command to print text
echo $ROOTDIR

# The variable can be used by using a $ sign 
# in front of the name
# echo is a command to print text
echo $ROOTDIR
/data/pt_02586/


: 1

Now I can build new variables by concatenating upon ROOTDIR  
I will create a SUBJECTDIR and write every future command with respect to subject directory, therefore simply changing SUBJECTDIR to another subject and re-running the same command without modification would process a new subject.

In [100]:
SUBJECTDIR=$ROOTDIR'sub_01/'
echo $SUBJECTDIR

SUBJECTDIR=$ROOTDIR'sub_01/'
echo $SUBJECTDIR
/data/pt_02586/sub_01/


: 1

A few last tricks:  
- Pressing the up and down arrow in a terminal cycles through the previously called command, no need to always re-type it.  
- pressing "tab" attempts to autocomplete file name, if there are more than one possibility, it will show you the subset of files and directories that fit.  
- you can always use "~" as a path pointing to your home directory ("cd ~") 
- you can use ".." to refer to the parent directory you are into. This is mainly used only to do "cd .." to go "up" by one folder, but can be mixed in the middle of a full path if needed.  
- if you just changed directory somewhere with "cd" you can do "cd -" to go back to where you were.  

### We can now explore a bit the environement and the data

In [101]:
# This command is specific to the environement at MPI CBS
# but its shows the software version I am using
# (what we are doing should work on more recent version too)
envlist

# This command is specific to the environement at MPI CBS
# but its shows the software version I am using
# (what we are doing should work on more recent version too)
envlist
mrtrix 3.0.0
fsl 5.0.11
ants 2.3.1


: 1

In [107]:
# /data/pt_02586 is our project storage at MPI CBS
cd /data/pt_02586

# /data/pt_02586 is our project storage at MPI CBS
cd /data/pt_02586


: 1

In [108]:
# "ll" is a shortcut for "ls -l"
# The subject are separated into folders
# The numbering is based on subject name alphabetical order
ll -thr

# "ll" is a shortcut for "ls -l"
# The subject are separated into folders
# The numbering is based on subject name alphabetical order
ll -thr
total 2.0G
-rwxrwx--- 1 anwander users 2.1G Nov 26 10:32 "DTI_analysis.rar"*
drwxrwx--- 3 anwander users    9 Nov 26 11:26 "sub_02"/
drwxrwx--- 3 anwander users    9 Nov 26 11:27 "sub_03"/
drwxrwx--- 3 anwander users    9 Nov 26 11:27 "sub_04"/
drwxrwx--- 3 anwander users    9 Nov 26 11:27 "sub_05"/
drwxrwx--- 3 anwander users    9 Nov 26 11:28 "sub_06"/
drwxrwx--- 3 anwander users    9 Nov 26 11:28 "sub_07"/
drwxrwx--- 3 anwander users    9 Nov 26 11:28 "sub_08"/
drwxrwx--- 3 anwander users    9 Nov 26 11:29 "sub_09"/
drwxrwx--- 3 anwander users    9 Nov 26 11:29 "sub_10"/
drwxrwx--- 3 anwander users    9 Nov 26 11:29 "sub_11"/
drwxrwx--- 3 anwander users    9 Nov 26 11:30 "sub_12"/
drwxrwx--- 3 anwander users    9 Nov 26 11:30 "sub_13"/
drwxrwx--- 3 anwander users    9 Nov 26 11:30 "sub_14"/
drwxrwx--- 3 anwander users    9 Nov 26 11:31 "sub_15

: 1

In [109]:
# we can check subject one
# There are a few key files:
# - a DICOM folder
# - a converted nifti image (nii.gz) for the T1 and for the DWI
# - json file for the scanning parameters of T1 and DWI
# - text file for the bvecs and the bvals
# - text file for the topup parameter (pe_table)
# - text file for the eddy files (config and ind)
ls sub_01/

# we can check subject one
# There are a few key files:
# - a DICOM folder
# - a converted nifti image (nii.gz) for the T1 and for the DWI
# - json file for the scanning parameters of T1 and DWI
# - text file for the bvecs and the bvals
# - text file for the topup parameter (pe_table)
# - text file for the eddy files (config and ind)
ls sub_01/
"bvals.txt"*
"bvecs.txt"*
"DICOM"/
"DTI_MP_64_2x2x2_s2_p2_6-8_20210910130531_9.bval"*
"DTI_MP_64_2x2x2_s2_p2_6-8_20210910130531_9.bvec"*
"DTI_MP_64_2x2x2_s2_p2_6-8_20210910130531_9.json"*
"DTI_MP_64_2x2x2_s2_p2_6-8_20210910130531_9.nii.gz"*
"eddy_config.txt"*
"eddy_ind.txt"*
"pe_table.txt"*
"preprocessing"/
"t1_mprage_sag_1x1x1_p2_64ch_20210910130531_3.json"*
"t1_mprage_sag_1x1x1_p2_64ch_20210910130531_3.nii.gz"*
"tmp.nii"*


: 1

In [110]:
# for each subject we mkdir a preprocessing folder
# to hold the large amount of intermediary data
cd sub_01/preprocessing

# for each subject we mkdir a preprocessing folder
# to hold the large amount of intermediary data
cd sub_01/preprocessing


: 1

In [111]:
# we can setup variable for important paths
ROOTDIR='/data/pt_02586/'
SUBJECTDIR=$ROOTDIR'sub_01/'

# we can setup variable for important paths
ROOTDIR='/data/pt_02586/'
SUBJECTDIR=$ROOTDIR'sub_01/'


: 1

### DICOM to NifTi conversion

In the begining, all we have is a DICOM folder, so we will go inspect what kind of data it contains.

#### mrinfo  
mrinfo is the "mri information" command from Mrtrix.  
It can be used on individual images or on entire DICOM directory.  
When using it on a directory, it will show us a menu to select which image from the directory we want the info.

There is a limitation in the bash notebook that I am using where I cannot run code that expect a user input (it will hang forever). Therefore I will copy paste the output of mrinfo to demonstrate instead

In [113]:
# This would be the command without the #
# mrinfo $SUBJECTDIR'DICOM'

# This would be the command without the #
# mrinfo $SUBJECTDIR'DICOM'


: 1

mrinfo: [done] scanning DICOM folder "/data/pt_02586/sub_01/DICOM"  
Select series ('q' to abort):  
   0 -  208 MR images 13:13:36 t1_mprage_sag_1x1x1_p2_64ch (*tfl3d1_16ns) [3] ORIGINAL PRIMARY M ND NORM  
   1 -   71 MR images 13:55:50 DTI_MP_64_2x2x2_s2_p2_6-8 (*ep_b1000#47) [9] ORIGINAL PRIMARY DIFFUSION NONE ND MOSAIC  

We see that we have 2 image series in the DICOM folder, number 0 is a T1 and number 1 is a DWI.  
Normally, you would press 0 or 1 in the terminal and get the info for the specific serie, here instead I will weird trick to emulate that (using echo with the series numbers 0 and 1 into the mrinfo command).

The command shows the data dimension in voxels, the voxel size in mm, the "relative data strides" (more later), the data type (more later) and the affine matrix (under transform). The rest is collection of mr parameters and notes.

In [114]:
# mrinfo for serie 0, the T1
echo 0 | mrinfo $SUBJECTDIR'DICOM'

# mrinfo for serie 0, the T1
echo 0 | mrinfo $SUBJECTDIR'DICOM'
mrinfo: [done] scanning DICOM folder "/data/pt_02586/sub_01/DICOM"
Select series ('q' to abort):
   0 -  208 MR images 13:13:36 t1_mprage_sag_1x1x1_p2_64ch (*tfl3d1_16ns) [3] ORIGINAL PRIMARY M ND NORM
   1 -   71 MR images 13:55:50 DTI_MP_64_2x2x2_s2_p2_6-8 (*ep_b1000#47) [9] ORIGINAL PRIMARY DIFFUSION NONE ND MOSAIC
mrinfo: [100%] reading DICOM series "t1_mprage_sag_1x1x1_p2_64ch"
************************************************
Image name:          "20210910_P02_FengLP_GCY_JiangChaoYong (20210910_P02_FengLP_GCY_JiangChaoYong) [MR] t1_mprage_sag_1x1x1_p2_64ch"
************************************************
  Dimensions:        208 x 256 x 256
  Voxel size:        1 x 1 x 1
  Data strides:      [ 3 -1 -2 ]
  Format:            DICOM
  Data type:         unsigned 16 bit integer (little endian)
  Intensity scaling: offset = 0, multiplier = 1
  Transform:               0.9997           0    -0.02443      -98.33
           

: 1

In [115]:
# mrinfo for serie 1, the DWI
echo 1 | mrinfo $SUBJECTDIR'DICOM'

# mrinfo for serie 1, the DWI
echo 1 | mrinfo $SUBJECTDIR'DICOM'
mrinfo: [done] scanning DICOM folder "/data/pt_02586/sub_01/DICOM"
Select series ('q' to abort):
   0 -  208 MR images 13:13:36 t1_mprage_sag_1x1x1_p2_64ch (*tfl3d1_16ns) [3] ORIGINAL PRIMARY M ND NORM
   1 -   71 MR images 13:55:50 DTI_MP_64_2x2x2_s2_p2_6-8 (*ep_b1000#47) [9] ORIGINAL PRIMARY DIFFUSION NONE ND MOSAIC
mrinfo: [100%] reading DICOM series "DTI_MP_64_2x2x2_s2_p2_6-8"
************************************************
Image name:          "20210910_P02_FengLP_GCY_JiangChaoYong (20210910_P02_FengLP_GCY_JiangChaoYong) [MR] DTI_MP_64_2x2x2_s2_p2_6-8"
************************************************
  Dimensions:        128 x 128 x 88 x 71
  Voxel size:        1.71875 x 1.71875 x 1.7 x ?
  Data strides:      [ -1 -2 3 4 ]
  Format:            DICOM
  Data type:         unsigned 16 bit integer (little endian)
  Intensity scaling: offset = 0, multiplier = 1
  Transform:                    1           0          -0   

: 1

Has expected, we see that the T1 has 3 dimensions and 1mm isotropic voxels while the DWI has 4 dimensions and 1.7 isotropic voxels.

#### strides
Strides are related to the way the multi-dimensional is "unwraped" into a long 1D sequence of data to be stored on memory.  

For instance, if you have a 2D table of numbers that needed to be "flattened" into 1D, you could stack end-to-end the rows of the table. Similarly, you could stack end-to-end the columns instead.  
Both of these approach are completely reasonable and ideologically equivalent, however they obviously result in a different arrangement of the numbers to be stored in memory.  

The strides are essentially what "encodes" the information on how to unravel (to be exact, the strides in mrinfo are relative and normalized strides, but thats not important here).  

My personal opinion is that it is good practice to always convert your data to strides (1,2,3,4), this way, if any software in your pipeline happens to ignore stride information (it exist), you data is already in the natural ordering and this won't be an issue.

#### data type
The numbers values saved by an MRI scanner in the images don't have an absolute meaning.  
The scanner measures essentially electrical voltages and discretize them into a fixed number of bins.  
It is therefore really common for mri to save data with a data type of some sort of integers.  
In practice, it is convinient to quickly convert those to float when we start doing manipulation, normalization, etc

#### mrconvert
mrconvert is the "mri conversion" command of mrtrix. It can convert DICOM to nifti. To do so, it uses the same interface as mrinfo, where it shows you all the images in the DICOM and you have to chose.  
It can also be used to change the strides, change the data type and extract various useful textfiles of relevant parameters from the DICOM.

For instance, if we use the following command and select the dwi data seris, we will extract the image to a file called dwi.nii.gz, with proper strides and data type.  
We will also extract the diffusion b-vectors and b-values.  
We also extract the "phase encoding" information (pe_table) which is required for the fsl-TOPUP preprocessing step.  
We also extract a slightly different formationg of the phase encoding (eddy config and indices) which is require for the fsl-EDDY preprocessing step.

In [117]:
# mrconvert DICOM/ \
#           dwi.nii.gz \
#           -strides 1,2,3,4 \
#           -datatype float32 \
#           -export_grad_fsl bvecs.txt bvals.txt \
#           -export_pe_table pe_table.txt \
#           -export_pe_eddy eddy_config.txt eddy_ind.txt

# mrconvert DICOM/ \
#           dwi.nii.gz \
#           -strides 1,2,3,4 \
#           -datatype float32 \
#           -export_grad_fsl bvecs.txt bvals.txt \
#           -export_pe_table pe_table.txt \
#           -export_pe_eddy eddy_config.txt eddy_ind.txt


: 1